In [3]:
import pandas as pd
import numpy as np
from datetime import datetime

In [4]:
# Đọc file CSV vào DataFrame
df = pd.read_csv('C:\\Users\\June\\Desktop\\archive (1)\\DataCoSupplyChainDataset.csv',encoding='latin-1')
df.head(5)

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,...,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Image,Product Name,Product Price,Product Status,shipping date (DateOrders),Shipping Mode
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,2/3/2018 22:56,Standard Class
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/18/2018 12:27,Standard Class
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/17/2018 12:06,Standard Class
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,Los Angeles,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/16/2018 11:45,Standard Class
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/15/2018 11:24,Standard Class


In [3]:
row_count, column_count = df.shape

print(f'Total rows: {row_count}')
print(f'Total columns: {column_count}')


Total rows: 180519
Total columns: 53


In [4]:
# Tìm tên cột có giá trị lỗi và số lượng giá trị lỗi của mỗi cột
columns_with_errors = []
error_counts = []

for column in df.columns:
    error_count = df[column].isna().sum()
    if error_count > 0:
        columns_with_errors.append(column)
        error_counts.append(error_count)

# In ra kết quả
if len(columns_with_errors) == 0:
    print("Không tìm thấy giá trị lỗi.")
else:
    for i in range(len(columns_with_errors)):
        print(f"Cột '{columns_with_errors[i]}' có {error_counts[i]} giá trị lỗi.")


Cột 'Customer Lname' có 8 giá trị lỗi.
Cột 'Customer Zipcode' có 3 giá trị lỗi.
Cột 'Order Zipcode' có 155679 giá trị lỗi.
Cột 'Product Description' có 180519 giá trị lỗi.


In [5]:
# Loại bỏ các cột có giá trị bị lỗi
df = df.dropna(subset=['Customer Lname','Customer Zipcode'])
df = df.drop(columns=['Order Zipcode','Product Description'])

In [6]:
df["order date (DateOrders)"] = pd.to_datetime(df["order date (DateOrders)"])
df["shipping date (DateOrders)"] = pd.to_datetime(df["shipping date (DateOrders)"])

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180508 entries, 0 to 180518
Data columns (total 51 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   Type                           180508 non-null  object        
 1   Days for shipping (real)       180508 non-null  int64         
 2   Days for shipment (scheduled)  180508 non-null  int64         
 3   Benefit per order              180508 non-null  float64       
 4   Sales per customer             180508 non-null  float64       
 5   Delivery Status                180508 non-null  object        
 6   Late_delivery_risk             180508 non-null  int64         
 7   Category Id                    180508 non-null  int64         
 8   Category Name                  180508 non-null  object        
 9   Customer City                  180508 non-null  object        
 10  Customer Country               180508 non-null  object        
 11  

In [7]:
# df = df[df['Order Status'].isin(['COMPLETE', 'PROCESSING'])]

#chia thành các cột ngày, tháng, năm
df['order day'] = df['order date (DateOrders)'].dt.strftime('%d')
df['order month'] = df['order date (DateOrders)'].dt.strftime('%m')
df['order quarter'] = (df['order month'].astype(int) - 1) // 3 + 1
df['order year'] = df['order date (DateOrders)'].dt.strftime('%Y')
df['shipping day'] = df['shipping date (DateOrders)'].dt.strftime('%d')
df['shipping month'] = df['shipping date (DateOrders)'].dt.strftime('%m')
df['shipping quarter'] = (df['shipping month'].astype(int) - 1) // 3 + 1
df['shipping year'] = df['shipping date (DateOrders)'].dt.strftime('%Y')
# Xóa cột "time" ban đầu khỏi DataFrame
df.drop('order date (DateOrders)', axis=1, inplace=True)
df.drop('shipping date (DateOrders)', axis=1, inplace=True)
# Xóa dữ liệu bị trùng và giữ lại hàng đầu tiên 
df.drop_duplicates(keep='first', inplace=True)

In [ ]:
#Tạo các bang
Order=df[['Type','Order Status','Order Id','order day','order month','order quarter','order year','Order Region','Order Item Cardprod Id',
         'Order State','Order Country','Order City','Customer Id']]
DetailOrder=df[['Order Item Total','Product Card Id','Sales','Order Item Id','Order Item Quantity',
                'Order Id','Benefit per order','Order Profit Per Order','Order Item Profit Ratio','Order Item Product Price']]
Delivery=df[['Order Id','Delivery Status','Shipping Mode','shipping day','shipping month','shipping quarter','shipping year',
             'Days for shipment (scheduled)','Days for shipping (real)','Late_delivery_risk']]
Customer = df[['Customer Id','Customer State','Customer Country','Customer City','Customer Street',
               'Customer Segment','Customer Lname','Customer Fname']]
Category=df[['Category Id','Category Name','Department Id']]

Product=df[['Product Name','Product Price','Product Category Id','Product Card Id','Category Id']]
Department=df[['Department Id','Department Name']]

In [ ]:
#xoa du lieu trung truoc khi luu
Order.drop_duplicates(subsetkeep='first', inplace=True)
DetailOrder.drop_duplicates(keep='first', inplace=True)
Delivery.drop_duplicates(keep='first', inplace=True)
Customer.drop_duplicates(keep='first', inplace=True)
Category.drop_duplicates(keep='first', inplace=True)
Product.drop_duplicates(keep='first', inplace=True)
Department.drop_duplicates(keep='first', inplace=True)

In [8]:


# Khởi tạo writer object và ghi các dataframe vào các sheet tương ứng
# Tạo ra một dictionary chứa các dim và tên của sheet tương ứng
data = {'Department':Department,'Product':Product,
        'Customer':Customer,
        'Category':Category,'Delivery': Delivery,
        'DetailOrder': DetailOrder,'Order':Order}




In [11]:
for sheet_name, df in data.items():
    df.to_csv(f'{sheet_name}.csv', index=False)